# Markov-Chain Monte-Carlo Methods 

\begin{align*}
\newcommand{\vec}[1]{\boldsymbol{#1}}
\newcommand{\pr}[1]{\Pr\left[#1\right]}
\newcommand{\p}[1]{p\left[#1\right]}
\newcommand{\SSS}{\mathcal{S}}
\end{align*}

* https://towardsdatascience.com/gibbs-sampling-8e4844560ae5
* https://stats.stackexchange.com/questions/48838/intuitive-explanation-for-periodicity-in-markov-chains
* https://bayes.wustl.edu/Manual/RadfordNeal.review.pdf
* http://www.mcmchandbook.net/HandbookChapter5.pdf
* https://www.jstor.org/stable/2289457#metadata_info_tab_contents
* http://hedibert.org/wp-content/uploads/2013/12/1987Pearl.pdf
* https://www.sciencedirect.com/science/article/abs/pii/0004370292900667
* https://books.google.ee/books?id=LHHrBwAAQBAJ&printsec=frontcover&hl=fr&source=gbs_ge_summary_r&cad=0#v=onepage&q&f=false

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability.python.mcmc import kernel as kernel_base


from pandas import DataFrame
from numpy.random import choice
from pretty_printing import mdisplay 

<center><h1>Markov Chain Monte Carlo in TFP</h1> </center>
<center><h2>Theoretical overview</h2></center>
<br>
<center><h3>Sven Laur</h3></center>
<center><h3>swen@ut.ee</h3></center>

## Markov Chains

For simplicity let us consider discrete Markov chain with three states. 
<center>
    <img src='./illustrations/three-state-model.png' width=40% alt='Markov Chain'>
</center>

* **State space:** The set of all possible states $\SSS=\{0,1,2\}$.  
* **Initial distribution:** Each row element $\pi_i=\pr{i}$ specifies the probability that we start form the state $i$. 
* **State transition matrix:** Each matrix element $p_{ij}=\pr{i\to j}$ specifies the probability going form the state $i$ to state $j$. 
* **Probability update rule:** The probability vector for next state is given $\pi(t+1)=\pi(t)P$


## Well-behaving  Markov Chain

Markov chain is irreversible iff each state can be reached from each other state.
<center>
    <img src='./illustrations/three-state-model.png' width=40% alt='Markov Chain'>
</center>

In [5]:
#pi = DataFrame([[1, 0, 0]])
pi = DataFrame([[1/3, 1/3, 1/3]])
P = DataFrame([[0, 1/2, 1/2], [1/2, 1/2, 0], [1/2, 0, 1/2]])
mdisplay([pi, P], ['Initial probability','Transition matrix'])
mdisplay([pi, pi.dot(P), pi.dot(P).dot(P), pi.dot(P).dot(P).dot(P), pi.dot(P).dot(P).dot(P).dot(P) ], ['t=0', 't=1', 't=2', 't=3', 't=4'])

Initial probability Transition matrix 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.0 
 0.5 
 0.5 
 
 
 0.5 
 0.5 
 0.0 
 
 
 0.5 
 0.0 
 0.5

t=0 t=1 t=2 t=3 t=4 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333

## Misbehaving Markov Chain

Markov Chain is absorbable if a state is not reachable from some other state.
<center>
    <img src='./illustrations/two-sink-model.png' width=40% alt='Absorbing Markov Chain'>
</center>


In [8]:
#pi = DataFrame([[1, 0, 0]])
#pi = DataFrame([[0, 1, 0]])
pi = DataFrame([[0, 0, 1]])
Q = DataFrame([[0, 1/2, 1/2], [0, 1, 0], [0, 0, 1]])
mdisplay([pi, Q], ['Initial probability','Transition matrix'])
mdisplay([pi, pi.dot(Q), pi.dot(Q).dot(Q), pi.dot(Q).dot(Q).dot(Q), pi.dot(Q).dot(Q).dot(Q).dot(Q) ], ['t=0', 't=1', 't=2', 't=3', 't=4'])

Initial probability Transition matrix 
 
 
 0 
 1 
 2 
 
 
 
 
 0 
 0 
 1 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0 
 0.5 
 0.5 
 
 
 0 
 1.0 
 0.0 
 
 
 0 
 0.0 
 1.0

t=0 t=1 t=2 t=3 t=4 
 
 
 0 
 1 
 2 
 
 
 
 
 0 
 0 
 1 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.0 
 0.0 
 1.0 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.0 
 0.0 
 1.0 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.0 
 0.0 
 1.0 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.0 
 0.0 
 1.0

## Misbehaving Markov Chain

* A time $t$ is a return-time for a state $x$ if it is possible to return to $x$ after $t$-timesteps.  
* A chain is aperiodic if after a finite time all return times are possible for each state.
* Periodic chains have sevaral stationary distributions. 
<center>
    <img src='./illustrations/periodic-chain.png' width=30% alt='Periodic chain'>
</center>

In [10]:
#pi = DataFrame([[1, 0, 0]])
#pi = DataFrame([[0, 1, 0]])
#pi = DataFrame([[0, 0, 1]])
pi = DataFrame([[1/3, 1/3, 1/3]])
Q = DataFrame([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
mdisplay([pi, Q], ['Initial probability','Transition matrix'])
mdisplay([pi, pi.dot(Q), pi.dot(Q).dot(Q), pi.dot(Q).dot(Q).dot(Q), pi.dot(Q).dot(Q).dot(Q).dot(Q) ], ['t=0', 't=1', 't=2', 't=3', 't=4'])

Initial probability Transition matrix 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0 
 1 
 0 
 
 
 0 
 0 
 1 
 
 
 1 
 0 
 0

t=0 t=1 t=2 t=3 t=4 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333 
 
 
 
 
 
 0 
 1 
 2 
 
 
 
 
 0.333333 
 0.333333 
 0.333333

##  Chains Monte-Carlo sampling

Ergodic Markov Chains have a uniqe stationary distribution and the chain converges to it regardless if the initial state.

**Theorem.** A Markov Chain is ergodic if and only if the chain is irreducible and aperiodic. 

**MCMC principle.** We can use Markov Chain for sor sampling provided that we can define state transitions so that the stationary distribution is our desired distribution. 

**Design goal**: We need to define transition probabilities so that for the transition preserves the desired distribution $\pi$: 

\begin{align*}
\pi[x]=\sum_y\pi[y]\cdot\p{y\to x}\enspace.
\end{align*}



##  Chains Monte-Carlo sampling


**Design goal.** We need to define transition probabilities so that for the transition preserves the desired distribution $\pi$: 

\begin{align*}
\pi[x]=\sum_y\pi[y]\cdot\p{y\to x}\enspace.
\end{align*}

There are three common ways to define sampling procedure:
* Gibbs sampling
* Metropolis–Hastings algorithm
* Hamiltonian MonteCarlo algorithm

## Gibbs sampler

**Problem.** We want to sample from the distribution $\p{x_1,\ldots, x_n}$ but we have learned only a conditional distributions $\p{x_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_n}$.

**State transition mechanism:**

\begin{align*}
x_1 &\sim \p{x_1|x_2,\ldots, x_n}\\
\ldots\\
x_i &\sim \p{x_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}}\\
\ldots\\
x_n &\sim \p{x_n|x_1\ldots,x_{n-1}}
\end{align*}

## Why Gibbs sampling works?

Note that the $i$-th subtransition preserves the distribution $\pi[\vec{x}]=\p{x_1,\ldots,x_n}$:

\begin{align*}
\sum_{\vec{y}} \pi(\vec{y})\cdot\p{\vec{y}\to \vec{x}}
&=\sum_{\vec{y}}\p{y_i|y_1,\ldots,y_{i-1},y_{i+1},\ldots,y_{n}}
\cdot\p{y_1,\ldots,y_{i-1},y_{i+1},\ldots,y_{n}} 
\cdot\p{x_i|y_1,\ldots,y_{i-1},y_{i+1},\ldots,y_{n}}\cdot \prod_{j\neq i} [x_j=y_j]\\
&=\sum_{y_i}\p{y_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}}
\cdot\p{x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}} 
\cdot\p{x_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}}\\
&=\p{x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}} 
\cdot\p{x_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}}
\cdot\sum_{y_i}\p{y_i|x_1,\ldots,x_{i-1},x_{i+1},\ldots,x_{n}}\\
&=\p{x_1,\ldots,x_{i-1},x_i,x_{i+1},\ldots,x_{n}}=\pi[\vec{x}] 
\end{align*}


## Why Gibbs sampling works?

Note that the $i$-th subtransition preserves the distribution $\pi[\vec{x}]=\p{x_1,\ldots,x_n}$:

\begin{align*}
\sum_{\vec{y}} \pi(\vec{y})\cdot\p{\vec{y}\to \vec{x}}
&=\p{x_1,\ldots,x_{i-1},x_i,x_{i+1},\ldots,x_{n}}=\pi[\vec{x}] 
\end{align*}

Consequently the entire transition must preserve the distribution. 

**Loopholes.** 
* The chain can be absorbing if some states of $x_i$ are not reachable for some sub-transitions.
* Additional care is needed when all states are not reachable from each sate.

## Simple example problem


Let us consider four element Markov field

<center>
<img src = 'illustrations/markov-random-field-iv.png' width=15%>
</center>    

where all random variales $X_i$ have only two states $\{0,1\}$ and the node is set to $1$ with the following probabilities
 

|Left |Right |Probability 
|---|---|---|
| 0 | 0 | $1/4$ |
| 0 | 1 | $1/4$ |
| 1 | 0 | $1/4$ |
| 1 | 1 | $3/4$ | 


## Naive solution in Pyhton

In [11]:
prob_map = {
    (0,0): 1/4, 
    (0,1): 1/4,
    (1,0): 1/4,
    (1,1): 3/4
}

In [12]:
for k in range(10):
    # Initialise Gibbs sampler 
    X = [0,1,0,1]
    for t in range(1000):
        # Singe state transition
        for i in range(4):
            # Sample from a probabilistic model
            p = prob_map[X[(i - 1) % 4], X[(i + 1) % 4]]
            X[i] = choice([0,1], p=[1-p, p])
    # Output a sample
    print(X)

[1, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 1, 1]
[0, 1, 1, 1]


## Naive implentation in Tensorflow Probability

In [13]:
for k in range(10):
    # Initialise Gibbs sampler 
    X = tf.constant([0,1,0,1]) 
    for t in range(1000):
        # Singe state transition
        for i in range(4):
            # Sample from probabilistic model
            y = X[(i - 1) % 4] & X[(i + 1) % 4]
            p = y * 3/4 + (1 - y) * 1/4
            x = int(np.random.uniform() <= p)
            # Update the state tensor
            X = tf.tensor_scatter_nd_update(X, [[i]], [x])
    # Output a sample
    print(list(X.numpy()))

[0, 1, 0, 0]
[0, 1, 1, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 1, 1]
[1, 0, 0, 1]
[0, 1, 0, 0]


## Canonical implementation using tfp.mcmc.TransitionKernel

**def** `bootstrap_results(init_state: Tensor) -> Tuple`: 
* Compute startup auxiliary information used in state update function. 

**def** `one_step(current_state: Tensor, auxiliary_info: Tuple) -> Tuple[Tensor, Tuple]`:
* Sample next state as Tensor. 
* Update auxiliary information used in next step

**def** `is_calibrated`
* Does it converge to specified distribution

To run the Markov Chain we need to invoke `tfp.mcmc.sample_chain` with parameters
* `num_results`
* `num_burnin_steps`
* `current_state`
* `kernel`

In [14]:
class GibbsKernel(kernel_base.TransitionKernel):
    
    def __init__(self):
        pass
    
    def is_calibrated(self):
        return True
    
    def bootstrap_results(self, init_state):
        return tuple()
    
    def one_step(self, X, auxiliary_info):
        y = X[(i - 1) % 4] & X[(i + 1) % 4]
        p = y * 3/4 + (1 - y) * 1/4
        x = int(np.random.uniform() <= p)
        # Update the state tensor
        return tf.tensor_scatter_nd_update(X, [[i]], [x]), tuple()  

In [15]:
output = tfp.mcmc.sample_chain(
    num_results=10,
    num_burnin_steps=1000,
    current_state=tf.constant([0,1,0,1]),
    kernel=GibbsKernel(),
    trace_fn=None
)
mdisplay([DataFrame(output.numpy(),columns=['X0', 'X1', 'X2', 'X3'])],['Output states'])

Output states 
 
 
 X0 
 X1 
 X2 
 X3 
 
 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 1 
 0 
 1 
 
 
 0 
 1 
 0 
 1 
 
 
 0 
 1 
 0 
 1 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 1 
 0 
 1 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 1 
 0 
 1 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 1 
 0 
 0

## Applications of Gibbs sampling

**Image synthesis:** 
* We can easily build an probabilistic model that predicts pixel form its neighbour region.

**Data Augmentation Algorithm:** 
* Sometimes it is impossible to implement the EM algorithm as the E-step does not have an analytical solution.  
* Gibbs sampling can bypass this problem and samples to approximate the expectation.

**Belief propagation in Bayesion networks**
* Sometimes analytical methods fail for Bayesian networks
* Gibbs sampling can be used to estimate posterior distribution.


**Bayesian training of neural networks** 

* The ancient articles explain how to train Bayesian neural network   

## Metropolis–Hastings algorithm

**Problem.** We want to sample from a distribution $\p{x_1,\ldots,x_n}\propto f(x_1,\ldots,x_n)$. 

**State transition mechanism:**

* Fix a proposal distribution $q[\vec{x}\to\vec{y}]$ for generating output candidates.
* Given a state $\vec{x}$ generate a proposal state $\vec{x}'$ according to the candidate transition.
* Accept $\vec{x}'$ with the probability 

  \begin{align*}
  a[\vec{x}\to\vec{x}']=\frac{\p{\smash{\vec{x}'}}}{\p{\vec{x}}}=\frac{f(\vec{x}')}{f(\vec{x})}\enspace.
  \end{align*}
* Otherwise, output $\vec{x}$. 

## Why Metropolis–Hastings algorithm works?

Note that detailed balance

  \begin{align*} 
  \pi[\vec{x}]\cdot\p{\vec{x}\to\vec{y}}=\pi[\vec{y}]\cdot\p{\vec{y}\to\vec{x}}
  \end{align*}

implies that the transition preserves the distribution

 \begin{align*}
 \sum_{\vec{y}}\pi[\vec{y}]\cdot \p{\vec{y}\to\vec{x}}=
 \sum_{\vec{y}}\pi[\vec{x}]\cdot \p{\vec{x}\to\vec{y}}=
 \pi[\vec{x}]\cdot\sum_{\vec{y}}\cdot \p{\vec{x}\to\vec{y}}=\pi[\vec{x}]
 \end{align*}

When $\vec{x}\neq \vec{y}$ we can express for the disribution $\p{\vec{x}}$

\begin{align*}
\p{\vec{x}}\cdot \p{\vec{x}\to\vec{y}}&=\p{\vec{x}}\cdot q[\vec{x}\to\vec{y}]\cdot a[\vec{x}\to\vec{y}]=q[\vec{x}\to\vec{y}]\cdot\min\{\p{\vec{y}},\p{\vec{x}}\}\\
\p{\vec{y}}\cdot \p{\vec{y}\to\vec{x}}&=\p{\vec{y}}\cdot q[\vec{y}\to\vec{x}]\cdot a[\vec{y}\to\vec{x}]=q[\vec{y}\to\vec{x}]\cdot\min\{\p{\vec{x}},\p{\vec{y}}\}
\end{align*}


When $\vec{x}\neq \vec{y}$ we can express for thedisribution $\p{\vec{x}}$

\begin{align*}
\p{\vec{x}}\cdot \p{\vec{x}\to\vec{y}}&=\p{\vec{x}}\cdot q[\vec{x}\to\vec{y}]\cdot a[\vec{x}\to\vec{y}]=q[\vec{x}\to\vec{y}]\cdot\min\{\p{\vec{y}},\p{\vec{x}}\}\\
\p{\vec{y}}\cdot \p{\vec{y}\to\vec{x}}&=\p{\vec{y}}\cdot q[\vec{y}\to\vec{x}]\cdot a[\vec{y}\to\vec{x}]=q[\vec{y}\to\vec{x}]\cdot\min\{\p{\vec{x}},\p{\vec{y}}\}
\end{align*}

Hence, the transition preserves the disribution $\p{\vec{x}}$ as soon as the proposal distribution is symmetric 

\begin{align*}
q[\vec{x}\to\vec{y}]=q[\vec{y}\to\vec{x}]\enspace.
\end{align*}


## Implementation using tfp.mcmc.MetropolisHastings

The kernel is fixed by specifying  `inner_kernel` with `inner_kernel.one_step()` function that returns auxiliary information in specific format:

* Output type is `collections.namedtuple`
* The tuple contains `target_log_prob` field
* The tuple mey contain `log_acceptance_correction` field
* Other Tensor type fields. 


## Detour to Hamiltonian MonteCarlo algorithm

**Problem.** We want to sample from a distribution $\p{x_1,\ldots,x_n}\propto f(x_1,\ldots,x_n)$. 

We can introduce artificial variables $y_1,\ldots,y_n$ and sample variables from the distribution

\begin{align*}
\p{\vec{x},\vec{y}}\propto\exp\left(-\frac{U(\vec{x})}{T}\right)\cdot\exp\left(-\frac{K(\vec{y})}{T}\right)
\end{align*}

where we can interpret 
* $\vec{x}$ as position variable and $\vec{y}$ as momentum variables 
* $U(\cdot)$ as potential and $K(\cdot)$ as kinetic energy.  
* $T$ as the temperature of the system

Now we are in the realm of statistical mechanics and molecular dynamics... 

## Hamiltonian as the total energy of the system

The standard notation for position variables $\vec{q}$ and for momentum variables $\vec{p}$ and the total energy of the system is defined

\begin{align*}
 H(\vec{q},\vec{p})=U(\vec{q})+K(\vec{p})
\end{align*}

Physical relation between $\vec{q},\vec{p}$ and energies yield a system of differential equations

\begin{align*}
\frac{dq_i}{dt}&=\frac{\partial H}{\partial p_i}\\
\frac{dp_i}{dt}&=-\frac{\partial H}{\partial q_i}
\end{align*}


Physical relation between $\vec{q},\vec{p}$ and energies yield a system of differential equations

\begin{align*}
\frac{dq_i}{dt}&=\frac{\partial H}{\partial p_i}\\
\frac{dp_i}{dt}&=-\frac{\partial H}{\partial q_i}
\end{align*}

which imply the invariant $ H(\vec{q},\vec{p})=const$ over time and introduces a determnistic transition $T$ over the phase space

\begin{align*}
(\vec{q},\vec{p})\xrightarrow{\quad\Delta t\ passes\quad} (\vec{q}',\vec{p}')
\end{align*}

## Properties of the deterministic transition

Hamiltonian dynamics assures that the transition $T$ has followinf properties
* The transition $T$ is invertible
* The transition preserves the extended distribution $\p{\vec{q},\vec{p}}$

As the transition is deterministic it is not a Markov chain that converges to $\p{\vec{q},\vec{p}}$.

* We must add a stohhastic component to momentum variables $\vec{p}$. 

##  Hamiltonian MonteCarlo algorithm

**Step 1**

* Draw momentum variables $\vec{p}$ according to the distribution

\begin{align*}
\frac{1}{Z_k}\exp\left(-\frac{K(\vec{p})}{T}\right)
\end{align*}

* For properly chosen kinetic energy function the corresponding distribution is a Gaussian noise without correlations.

**Step 2** 
* Apply the transition $T$ to the state $(\vec{q},\vec{p})$. 
* Let $(\vec{q}^*,\vec{p}^*)$ be the resulting state.
* Accept $(\vec{q}^*,\vec{p}^*)$ with the probability
  
  \begin{align*}
  \frac{\p{\vec{q}^*,\vec{p}^*}}{\p{\vec{q},\vec{p}}}=\exp\Bigl(H(\vec{q},\vec{p})-H(\vec{q}^*,\vec{p}^*)\Bigr)
  \end{align*}
  
* Otherwise keep the current state $(\vec{q},\vec{p})$.   

## Tricks in numerical integration of differential equations

Euler’s method for solving diferential equations $f(x+\varepsilon)= f(x) + f'(x)\cdot \varepsilon$ does not work well for solving Hamiltonian dynamics.

* It does not preserve total energy $H(\vec{q},\vec{p})$ and thus result becomes unstable.

The Leapfrog Method is much more stable and preserves total energy ($T=1$)

\begin{align*}
p_i(t+\varepsilon/2)&=p_i(t)−\frac{\partial U(q(t))}{\partial q_i}\cdot \frac{\varepsilon}{2}\\
q_i(t + \varepsilon) &= q_i(t) +\frac{\partial K(pi(t + ε/2))}{\partial p_i}\cdot \varepsilon\\
p_i(t+\varepsilon)&=p_i(t+\varepsilon/2)− \frac{∂U(q(t+ε))}{\partial q_i}\cdot \frac{\varepsilon}{2}.
\end{align*}


The approximate transformation is fixed by the step size $\varepsilon$ and the number of leapfrog steps.

## Implementation using tfp.mcmc.HamiltonianMonteCarlo


The kernel is specified by giving the following arguments:

* `target_log_prob_fn`: energy function for position variables
* `step_size`: step size for numerical integration of Hamiltonian dynamics
* `num_leapfrog_steps`: number of steps (time) for evaluating Hamiltonian.

Everything else is done behind the scenes.